# Experimentos

In [1]:
# imports
import numpy as np
import pandas as pd

# time
import time
from datetime import timedelta

# Metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

# ETL
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Oversampling Methods
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

# Deep Learning Framework - Keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, AveragePooling2D, Flatten
from keras.utils import to_categorical
from keras.backend import clear_session
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

# K-fold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

models = pd.DataFrame()

Using TensorFlow backend.


#  Raw Data

In [2]:
## Data
df = pd.read_csv('data/train.csv')

## Remove duplicate rows
df.drop_duplicates(inplace=True)

# Slit (X,Y)
x_data = df.drop(['ID','TARGET'], axis=1)
y_data = df['TARGET'].copy()

x_data.shape

(76020, 369)

# Oversampling: Random Oversampling

## 2 CNN 1 AVG RO

In [3]:
# info
model_name = '2 CNN 1 AVG RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97590 samples, validate on 13684 samples
Epoch 1/10000
97590/97590 [==============================] - 38s - loss: 0.5761 - acc: 0.7069 - val_loss: 0.5783 - val_acc: 0.7178
Epoch 2/10000
97590/97590 [==============================] - 18s - loss: 0.5109 - acc: 0.7565 - val_loss: 0.4266 - val_acc: 0.8039
Epoch 3/10000
97590/97590 [==============================] - 19s - loss: 0.4899 - acc: 0.7661 - val_loss: 0.4736 - val_acc: 0.7629
Epoch 4/10000
97590/97590 [==============================] - 19s - loss: 0.4705 - acc: 0.7771 - val_loss: 0.5545 - val_acc: 0.7274
Epoch 5/10000
97590/97590 [==============================] - 18s - loss: 0.4564 - acc: 0.7846 - val_loss: 0.4785 - val_acc: 0.7602
Epoch 6/10000
97590/97590 [==============================] - 18s - loss: 0.4438 - acc: 0.7922 - val_loss: 0.4154 - val_acc: 0.7873
Epoch 7/10000
97590/97590 [==============================] - 19s - loss: 0.4305 - acc: 0.7981 - val_loss: 0.4805 - val_acc: 0.7496
Epoch 8/10000
97590/97590 [=

97572/97572 [==============================] - 18s - loss: 0.2879 - acc: 0.8751 - val_loss: 0.4554 - val_acc: 0.8150
Epoch 40/10000
97572/97572 [==============================] - 18s - loss: 0.2942 - acc: 0.8743 - val_loss: 0.4934 - val_acc: 0.8107
Epoch 41/10000
97572/97572 [==============================] - 18s - loss: 0.2899 - acc: 0.8765 - val_loss: 0.4530 - val_acc: 0.8533
Epoch 42/10000
97572/97572 [==============================] - 18s - loss: 0.3013 - acc: 0.8760 - val_loss: 0.5083 - val_acc: 0.8196
Epoch 43/10000
97572/97572 [==============================] - 18s - loss: 0.2982 - acc: 0.8763 - val_loss: 0.4438 - val_acc: 0.8423
Epoch 44/10000
97572/97572 [==============================] - 18s - loss: 0.2964 - acc: 0.8767 - val_loss: 0.4608 - val_acc: 0.8279
Epoch 45/10000
97572/97572 [==============================] - 19s - loss: 0.2895 - acc: 0.8795 - val_loss: 0.4426 - val_acc: 0.8396
Epoch 46/10000
97572/97572 [==============================] - 18s - loss: 0.2912 - acc: 0.8

97450/97450 [==============================] - 18s - loss: 0.4059 - acc: 0.8118 - val_loss: 0.4793 - val_acc: 0.7458
Epoch 11/10000
97450/97450 [==============================] - 18s - loss: 0.3951 - acc: 0.8160 - val_loss: 0.4489 - val_acc: 0.7955
Epoch 12/10000
97450/97450 [==============================] - 18s - loss: 0.3935 - acc: 0.8196 - val_loss: 0.4927 - val_acc: 0.7639
Epoch 13/10000
97450/97450 [==============================] - 18s - loss: 0.3820 - acc: 0.8249 - val_loss: 0.3935 - val_acc: 0.8217
Epoch 14/10000
97450/97450 [==============================] - 18s - loss: 0.3754 - acc: 0.8295 - val_loss: 0.4500 - val_acc: 0.7895
Epoch 15/10000
97450/97450 [==============================] - 18s - loss: 0.3686 - acc: 0.8329 - val_loss: 0.3763 - val_acc: 0.8343
Epoch 16/10000
97450/97450 [==============================] - 18s - loss: 0.3607 - acc: 0.8364 - val_loss: 0.4202 - val_acc: 0.8215
Epoch 17/10000
97450/97450 [==============================] - 18s - loss: 0.3604 - acc: 0.8

97468/97468 [==============================] - 18s - loss: 0.3602 - acc: 0.8368 - val_loss: 0.4940 - val_acc: 0.7666
Epoch 18/10000
97468/97468 [==============================] - 18s - loss: 0.3546 - acc: 0.8399 - val_loss: 0.4631 - val_acc: 0.7865
Epoch 19/10000
97468/97468 [==============================] - 18s - loss: 0.3498 - acc: 0.8414 - val_loss: 0.4586 - val_acc: 0.8032
Epoch 20/10000
97468/97468 [==============================] - 18s - loss: 0.3479 - acc: 0.8435 - val_loss: 0.4813 - val_acc: 0.7827
Epoch 21/10000
97468/97468 [==============================] - 18s - loss: 0.3415 - acc: 0.8467 - val_loss: 0.4454 - val_acc: 0.8000
Epoch 22/10000
97468/97468 [==============================] - 18s - loss: 0.3392 - acc: 0.8469 - val_loss: 0.4934 - val_acc: 0.8063
Epoch 23/10000
97468/97468 [==============================] - 18s - loss: 0.3373 - acc: 0.8483 - val_loss: 0.4859 - val_acc: 0.7917
Epoch 24/10000
97468/97468 [==============================] - 18s - loss: 0.3320 - acc: 0.8

In [4]:
print(model_name)

metrics_kf

2 CNN 1 AVG RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.957366,0.708977,0.708977,0.551553,0.689363,0.555344,0.834671,00:15:59.838693
KF_2,0.937666,0.769982,0.769982,0.555619,0.736737,0.553092,0.809417,00:10:14.863899
KF_3,0.954528,0.736393,0.736393,0.554732,0.704663,0.559302,0.833465,00:14:25.158268
KF_4,0.942323,0.769694,0.769694,0.554075,0.719153,0.553985,0.818469,00:08:24.818699
KF_5,0.955111,0.730715,0.730715,0.547787,0.697525,0.543205,0.810576,00:14:31.307570
KF_6,0.948960,0.752149,0.752149,0.556614,0.713876,0.561648,0.832807,00:08:30.934915
KF_7,0.946444,0.759289,0.759289,0.552377,0.714392,0.550793,0.815443,00:09:15.664918
KF_8,0.948174,0.724571,0.724571,0.553041,0.681251,0.560418,0.846619,00:15:17.110484
KF_9,0.946181,0.752281,0.752281,0.559467,0.710238,0.568894,0.845547,00:10:26.643074
KF_10,0.938537,0.747755,0.747755,0.554525,0.724007,0.553987,0.816735,00:08:04.589005


## 2 CNN 2 AVG RO

In [5]:
# info
model_name = '2 CNN 2 AVG RO'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = RandomOverSampler(random_state=42)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97536 samples, validate on 13684 samples
Epoch 1/10000
97536/97536 [==============================] - 20s - loss: 0.6727 - acc: 0.6003 - val_loss: 0.6246 - val_acc: 0.7078
Epoch 2/10000
97536/97536 [==============================] - 20s - loss: 0.6248 - acc: 0.6762 - val_loss: 0.5964 - val_acc: 0.7003
Epoch 3/10000
97536/97536 [==============================] - 20s - loss: 0.6055 - acc: 0.6873 - val_loss: 0.5630 - val_acc: 0.6763
Epoch 4/10000
97536/97536 [==============================] - 20s - loss: 0.5595 - acc: 0.7163 - val_loss: 0.5488 - val_acc: 0.7316
Epoch 5/10000
97536/97536 [==============================] - 20s - loss: 0.5358 - acc: 0.7334 - val_loss: 0.5514 - val_acc: 0.6855
Epoch 6/10000
97536/97536 [==============================] - 20s - loss: 0.5282 - acc: 0.7348 - val_loss: 0.4951 - val_acc: 0.7394
Epoch 7/10000
97536/97536 [==============================] - 20s - loss: 0.5232 - acc: 0.7385 - val_loss: 0.5344 - val_acc: 0.7061
Epoch 8/10000
97536/97536 [=

97610/97610 [==============================] - 19s - loss: 0.4859 - acc: 0.7678 - val_loss: 0.5175 - val_acc: 0.7335
Epoch 28/10000
97610/97610 [==============================] - 19s - loss: 0.4845 - acc: 0.7694 - val_loss: 0.4630 - val_acc: 0.7670
Epoch 29/10000
97610/97610 [==============================] - 19s - loss: 0.4821 - acc: 0.7707 - val_loss: 0.4864 - val_acc: 0.7518
Epoch 30/10000
97610/97610 [==============================] - 19s - loss: 0.4791 - acc: 0.7711 - val_loss: 0.4750 - val_acc: 0.7473
Epoch 31/10000
97610/97610 [==============================] - 20s - loss: 0.4781 - acc: 0.7726 - val_loss: 0.4524 - val_acc: 0.7556
Epoch 32/10000
97610/97610 [==============================] - 19s - loss: 0.4765 - acc: 0.7730 - val_loss: 0.5354 - val_acc: 0.7246
Epoch 33/10000
97610/97610 [==============================] - 20s - loss: 0.4756 - acc: 0.7730 - val_loss: 0.4795 - val_acc: 0.7471
Epoch 34/10000
97610/97610 [==============================] - 19s - loss: 0.4731 - acc: 0.7

97438/97438 [==============================] - 20s - loss: 0.5027 - acc: 0.7588 - val_loss: 0.4692 - val_acc: 0.7720
Epoch 12/10000
97438/97438 [==============================] - 20s - loss: 0.5002 - acc: 0.7583 - val_loss: 0.4916 - val_acc: 0.7541
Epoch 13/10000
97438/97438 [==============================] - 19s - loss: 0.4974 - acc: 0.7611 - val_loss: 0.5115 - val_acc: 0.7168
Epoch 14/10000
97438/97438 [==============================] - 19s - loss: 0.4949 - acc: 0.7621 - val_loss: 0.5285 - val_acc: 0.7224
Epoch 15/10000
97438/97438 [==============================] - 19s - loss: 0.4927 - acc: 0.7632 - val_loss: 0.5060 - val_acc: 0.7416
Epoch 16/10000
97438/97438 [==============================] - 19s - loss: 0.4904 - acc: 0.7648 - val_loss: 0.5566 - val_acc: 0.6991
Epoch 17/10000
97438/97438 [==============================] - 19s - loss: 0.4888 - acc: 0.7660 - val_loss: 0.5076 - val_acc: 0.7488
Epoch 18/10000
97438/97438 [==============================] - 19s - loss: 0.4867 - acc: 0.7

97658/97658 [==============================] - 19s - loss: 0.4886 - acc: 0.7607 - val_loss: 0.4790 - val_acc: 0.7635
Epoch 22/10000
97658/97658 [==============================] - 19s - loss: 0.4865 - acc: 0.7628 - val_loss: 0.4631 - val_acc: 0.7534
Epoch 23/10000
97658/97658 [==============================] - 19s - loss: 0.4863 - acc: 0.7636 - val_loss: 0.6170 - val_acc: 0.6848
Epoch 24/10000
97658/97658 [==============================] - 20s - loss: 0.4821 - acc: 0.7655 - val_loss: 0.5195 - val_acc: 0.7294
Epoch 25/10000
97658/97658 [==============================] - 20s - loss: 0.4812 - acc: 0.7639 - val_loss: 0.4942 - val_acc: 0.7474
Epoch 26/10000
97658/97658 [==============================] - 19s - loss: 0.4800 - acc: 0.7662 - val_loss: 0.5271 - val_acc: 0.7466
Epoch 27/10000
97658/97658 [==============================] - 20s - loss: 0.4789 - acc: 0.7671 - val_loss: 0.5139 - val_acc: 0.7577
Epoch 28/10000
97658/97658 [==============================] - 20s - loss: 0.4770 - acc: 0.7

In [6]:
print(model_name)

metrics_kf

2 CNN 2 AVG RO


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.867820,0.798907,0.798907,0.543510,0.732677,0.510190,0.734315,00:10:16.961625
KF_2,0.853529,0.831659,0.834981,0.545709,0.761933,0.500506,0.704853,00:11:37.963512
KF_3,0.859004,0.815292,0.815292,0.544682,0.738395,0.512292,0.736122,00:09:57.718067
KF_4,0.893634,0.788096,0.788096,0.548286,0.732431,0.531113,0.773612,00:23:28.913460
KF_5,0.890051,0.783014,0.783014,0.541248,0.715584,0.510240,0.741252,00:22:05.969790
KF_6,0.883283,0.793986,0.793986,0.542965,0.731152,0.508384,0.731386,00:15:33.153000
KF_7,0.889856,0.802521,0.805844,0.548813,0.743152,0.528007,0.763615,00:21:29.008844
KF_8,0.864117,0.785926,0.785926,0.543161,0.730021,0.510187,0.735333,00:11:53.381164
KF_9,0.874450,0.809380,0.809380,0.545730,0.737734,0.518135,0.747928,00:16:52.907233
KF_10,0.861723,0.808513,0.808513,0.542728,0.746505,0.495205,0.700303,00:11:54.687280


# Oversampling: SMOTE

## 2 CNN 1 AVG SM

In [7]:
# info
model_name = '2 CNN 1 AVG SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(243, activation='relu'))
    model.add(Dense(243, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97569 samples, validate on 13684 samples
Epoch 1/10000
97569/97569 [==============================] - 19s - loss: 0.4807 - acc: 0.7860 - val_loss: 0.2785 - val_acc: 0.8779
Epoch 2/10000
97569/97569 [==============================] - 18s - loss: 0.3480 - acc: 0.8569 - val_loss: 0.2984 - val_acc: 0.8525
Epoch 3/10000
97569/97569 [==============================] - 18s - loss: 0.3166 - acc: 0.8707 - val_loss: 0.4519 - val_acc: 0.8072
Epoch 4/10000
97569/97569 [==============================] - 18s - loss: 0.2992 - acc: 0.8787 - val_loss: 0.3430 - val_acc: 0.8411
Epoch 5/10000
97569/97569 [==============================] - 18s - loss: 0.2896 - acc: 0.8834 - val_loss: 0.3023 - val_acc: 0.8625
Epoch 6/10000
97569/97569 [==============================] - 18s - loss: 0.2788 - acc: 0.8883 - val_loss: 0.3605 - val_acc: 0.8475
Epoch 7/10000
97569/97569 [==============================] - 18s - loss: 0.2699 - acc: 0.8921 - val_loss: 0.2881 - val_acc: 0.8666
Epoch 8/10000
97569/97569 [=

97617/97617 [==============================] - 18s - loss: 0.2103 - acc: 0.9165 - val_loss: 0.2914 - val_acc: 0.8844
Epoch 39/10000
97617/97617 [==============================] - 18s - loss: 0.2082 - acc: 0.9184 - val_loss: 0.3606 - val_acc: 0.8457
Epoch 40/10000
97617/97617 [==============================] - 18s - loss: 0.2071 - acc: 0.9188 - val_loss: 0.3134 - val_acc: 0.8737
Epoch 41/10000
97617/97617 [==============================] - 18s - loss: 0.2072 - acc: 0.9183 - val_loss: 0.3277 - val_acc: 0.8723
Epoch 42/10000
97600/97617 [============================>.] - ETA: 0sKF_5
Train on 97515 samples, validate on 13684 samples
Epoch 1/10000
97515/97515 [==============================] - 18s - loss: 0.4836 - acc: 0.7823 - val_loss: 0.3339 - val_acc: 0.8552
Epoch 2/10000
97515/97515 [==============================] - 18s - loss: 0.3533 - acc: 0.8501 - val_loss: 0.2890 - val_acc: 0.8709
Epoch 3/10000
97515/97515 [==============================] - 18s - loss: 0.3206 - acc: 0.8660 - val_l

97595/97595 [==============================] - 18s - loss: 0.2223 - acc: 0.9121 - val_loss: 0.2841 - val_acc: 0.8864
Epoch 23/10000
97595/97595 [==============================] - 18s - loss: 0.2205 - acc: 0.9127 - val_loss: 0.3344 - val_acc: 0.8680
Epoch 24/10000
97595/97595 [==============================] - 18s - loss: 0.2183 - acc: 0.9132 - val_loss: 0.3302 - val_acc: 0.8679
Epoch 25/10000
97595/97595 [==============================] - 18s - loss: 0.2166 - acc: 0.9140 - val_loss: 0.3541 - val_acc: 0.8628
Epoch 26/10000
97595/97595 [==============================] - 18s - loss: 0.2143 - acc: 0.9148 - val_loss: 0.3700 - val_acc: 0.8509
Epoch 27/10000
97595/97595 [==============================] - 18s - loss: 0.2142 - acc: 0.9161 - val_loss: 0.2810 - val_acc: 0.8983
Epoch 28/10000
97595/97595 [==============================] - 18s - loss: 0.2127 - acc: 0.9158 - val_loss: 0.3680 - val_acc: 0.8543
Epoch 29/10000
97595/97595 [==============================] - 18s - loss: 0.2099 - acc: 0.9

In [8]:
print(model_name)

metrics_kf

2 CNN 1 AVG SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.972858,0.774308,0.774308,0.564973,0.672468,0.582806,0.881757,00:08:57.184992
KF_2,0.972582,0.761031,0.764353,0.570750,0.706926,0.590182,0.874523,00:09:31.176331
KF_3,0.970616,0.776980,0.776980,0.552840,0.711874,0.552757,0.819784,00:07:24.711519
KF_4,0.975755,0.776916,0.776916,0.573341,0.707654,0.594301,0.878979,00:12:51.894413
KF_5,0.972661,0.764153,0.764153,0.561463,0.695018,0.575071,0.860826,00:13:12.359244
KF_6,0.979718,0.754317,0.754317,0.572477,0.700649,0.593249,0.880821,00:15:57.162566
KF_7,0.976632,0.767096,0.770418,0.570671,0.705599,0.590134,0.875033,00:11:58.068241
KF_8,0.971372,0.748836,0.748836,0.566745,0.680014,0.585225,0.880952,00:08:53.815671
KF_9,0.968531,0.790413,0.793746,0.566855,0.716789,0.582582,0.861202,00:06:44.783743
KF_10,0.971162,0.768955,0.768955,0.558805,0.714097,0.566736,0.840679,00:08:54.403232


## 2 CNN 2 AVG SM

In [9]:
# info
model_name = '2 CNN 2 AVG SM'

i = 1
metrics_kf = pd.DataFrame()
for train, test in kfold.split(x_data, y_data):

    ## Data
    x_train = x_data.iloc[train]
    y_train = y_data.iloc[train]
    
    x_test = x_data.iloc[test]
    y_test = y_data.iloc[test]
    
    ## Train, validation
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)
    
    ## Remove duplicate label (Train)
    removeIndex = x_train[x_train.duplicated(keep=False)].index
    x_train = x_train.drop(removeIndex)
    y_train = y_train.drop(removeIndex)
    
    ## Standard Scaler
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    x_val = scaler.transform(x_val)
    
    # Oversampling
    sampling = SMOTE(kind='borderline2',k_neighbors=5, random_state=42, n_jobs=4)
    x_train, y_train = sampling.fit_sample(x_train, y_train)
    
    # one hot
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)
    
    ## New Shape 20x20
    # Train
    n = x_train.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_train = np.concatenate((x_train, zeros), axis=1)
    
    # Val
    n = x_val.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_val = np.concatenate((x_val, zeros), axis=1)
    
    # Test
    n = x_test.shape[0]
    zeros = np.zeros((n,31),np.int32)
    x_test = np.concatenate((x_test, zeros), axis=1)
    
    # reshape to be [samples][width][height][pixels]
    x_train = x_train.reshape(x_train.shape[0], 20, 20, 1)
    x_test = x_test.reshape(x_test.shape[0], 20, 20, 1)
    x_val = x_val.reshape(x_val.shape[0], 20, 20, 1)
    
    ## status
    kf_i = 'KF_{0}'.format(i)
    print(kf_i)
    
    ## Train
    startTime = time.time()
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, input_shape=(20, 20, 1), activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(Conv2D(filters=6, kernel_size=(2,2), strides=1, activation='relu'))
    model.add(AveragePooling2D(pool_size=(2,2), strides=2))
    model.add(Flatten())
    model.add(Dense(94, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x=x_train,
          y=y_train,
          batch_size=128, 
          epochs=10000, 
          callbacks=[EarlyStopping(patience=20)], 
          validation_data=(x_val, y_val))
    tm = timedelta(seconds=(time.time()-startTime))
    
    ## Predict
    y_pred_proba = model.predict_proba(x_test)
    y_pred = y_pred_proba.argmax(axis=1)
    
    ## Metrics
    auc_train = roc_auc_score(y_train[:,1], model.predict_proba(x_train)[:,1])
    fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_proba[:,1])
    auc_1 = auc(fpr, tpr)
    auc_0 = auc(1-tpr, 1-fpr)
    precision = precision_score(y_test.values, y_pred, average='macro')
    recall = recall_score(y_test.values, y_pred, average='macro')  
    f1 = f1_score(y_test.values, y_pred, average='macro')
    accuracy = accuracy_score(y_test.values, y_pred)
           
    metrics_kf.loc[kf_i,'AURoc Train'] = auc_train
    metrics_kf.loc[kf_i,'AURoc 0'] = auc_0
    metrics_kf.loc[kf_i,'AURoc 1'] = auc_1
    metrics_kf.loc[kf_i,'Precision'] = precision
    metrics_kf.loc[kf_i,'Recall'] = recall
    metrics_kf.loc[kf_i,'F1 score'] = f1
    metrics_kf.loc[kf_i,'Accuracy'] = accuracy
    metrics_kf.loc[kf_i,'Time'] = tm
    if i == 1:
        plot_model(model, to_file='models/model_' + model_name.replace(' ', '_') + '.png', show_shapes=True)
    i=i+1
    clear_session()

print()
for m in ['AURoc Train', 'AURoc 0', 'AURoc 1', 'Precision', 'Recall', 'F1 score', 'Accuracy', 'Time']:

    mean = metrics_kf[m].mean()
    metrics_kf.loc['Media', m] = mean
    models.loc[model_name, m] = mean
    
    metrics_kf.loc['STD', m] = metrics_kf[m].std()

KF_1
Train on 97539 samples, validate on 13684 samples
Epoch 1/10000
97539/97539 [==============================] - 20s - loss: 0.6115 - acc: 0.6943 - val_loss: 0.5739 - val_acc: 0.7227
Epoch 2/10000
97539/97539 [==============================] - 19s - loss: 0.5313 - acc: 0.7661 - val_loss: 0.6112 - val_acc: 0.6771
Epoch 3/10000
97539/97539 [==============================] - 20s - loss: 0.5085 - acc: 0.7757 - val_loss: 0.3369 - val_acc: 0.7673
Epoch 4/10000
97539/97539 [==============================] - 20s - loss: 0.4691 - acc: 0.7891 - val_loss: 0.4128 - val_acc: 0.7173
Epoch 5/10000
97539/97539 [==============================] - 20s - loss: 0.4451 - acc: 0.7986 - val_loss: 0.4271 - val_acc: 0.7564
Epoch 6/10000
97539/97539 [==============================] - 20s - loss: 0.4284 - acc: 0.8050 - val_loss: 0.3269 - val_acc: 0.9183
Epoch 7/10000
97539/97539 [==============================] - 19s - loss: 0.4174 - acc: 0.8080 - val_loss: 0.2756 - val_acc: 0.9294
Epoch 8/10000
97539/97539 [=

97555/97555 [==============================] - 19s - loss: 0.3400 - acc: 0.8559 - val_loss: 0.4289 - val_acc: 0.8053
Epoch 13/10000
97555/97555 [==============================] - 20s - loss: 0.3366 - acc: 0.8578 - val_loss: 0.3472 - val_acc: 0.8445
Epoch 14/10000
97555/97555 [==============================] - 20s - loss: 0.3333 - acc: 0.8603 - val_loss: 0.4942 - val_acc: 0.7645
Epoch 15/10000
97555/97555 [==============================] - 20s - loss: 0.3292 - acc: 0.8619 - val_loss: 0.3166 - val_acc: 0.8550
Epoch 16/10000
97555/97555 [==============================] - 19s - loss: 0.3290 - acc: 0.8625 - val_loss: 0.3542 - val_acc: 0.8344
Epoch 17/10000
97555/97555 [==============================] - 20s - loss: 0.3257 - acc: 0.8633 - val_loss: 0.3884 - val_acc: 0.8290
Epoch 18/10000
97555/97555 [==============================] - 20s - loss: 0.3216 - acc: 0.8653 - val_loss: 0.2894 - val_acc: 0.8700
Epoch 19/10000
97555/97555 [==============================] - 20s - loss: 0.3200 - acc: 0.8

97543/97543 [==============================] - 19s - loss: 0.3263 - acc: 0.8631 - val_loss: 0.2492 - val_acc: 0.8807
Epoch 25/10000
97543/97543 [==============================] - 19s - loss: 0.3273 - acc: 0.8632 - val_loss: 0.3444 - val_acc: 0.8479
Epoch 26/10000
97543/97543 [==============================] - 19s - loss: 0.3237 - acc: 0.8636 - val_loss: 0.3229 - val_acc: 0.8620
Epoch 27/10000
97543/97543 [==============================] - 20s - loss: 0.3238 - acc: 0.8636 - val_loss: 0.3189 - val_acc: 0.8549
Epoch 28/10000
97543/97543 [==============================] - 19s - loss: 0.3223 - acc: 0.8655 - val_loss: 0.4018 - val_acc: 0.8271
Epoch 29/10000
97543/97543 [==============================] - 19s - loss: 0.3208 - acc: 0.8655 - val_loss: 0.3159 - val_acc: 0.8602
Epoch 30/10000
97408/97543 [============================>.] - ETA: 0sKF_5
Train on 97512 samples, validate on 13684 samples
Epoch 1/10000
97512/97512 [==============================] - 20s - loss: 0.6166 - acc: 0.6857 - val

97467/97467 [==============================] - 20s - loss: 0.2865 - acc: 0.8845 - val_loss: 0.3449 - val_acc: 0.8577
Epoch 58/10000
97467/97467 [==============================] - 20s - loss: 0.2841 - acc: 0.8848 - val_loss: 0.3203 - val_acc: 0.8593
Epoch 59/10000
97467/97467 [==============================] - 20s - loss: 0.2852 - acc: 0.8849 - val_loss: 0.4407 - val_acc: 0.8069
Epoch 60/10000
97467/97467 [==============================] - 19s - loss: 0.2843 - acc: 0.8859 - val_loss: 0.3351 - val_acc: 0.8554
Epoch 61/10000
97467/97467 [==============================] - 20s - loss: 0.2835 - acc: 0.8858 - val_loss: 0.3122 - val_acc: 0.8586
Epoch 62/10000
97467/97467 [==============================] - 20s - loss: 0.2827 - acc: 0.8853 - val_loss: 0.3491 - val_acc: 0.8429
Epoch 63/10000
97467/97467 [==============================] - 20s - loss: 0.2830 - acc: 0.8849 - val_loss: 0.3012 - val_acc: 0.8622
Epoch 64/10000
97467/97467 [==============================] - 19s - loss: 0.2837 - acc: 0.8

97519/97519 [==============================] - 20s - loss: 0.3012 - acc: 0.8767 - val_loss: 0.3304 - val_acc: 0.8503
Epoch 44/10000
97519/97519 [==============================] - 20s - loss: 0.2965 - acc: 0.8783 - val_loss: 0.3629 - val_acc: 0.8397
Epoch 45/10000
97519/97519 [==============================] - 20s - loss: 0.2961 - acc: 0.8787 - val_loss: 0.3255 - val_acc: 0.8537
Epoch 46/10000
97519/97519 [==============================] - 20s - loss: 0.2989 - acc: 0.8776 - val_loss: 0.3265 - val_acc: 0.8612
Epoch 47/10000
97519/97519 [==============================] - 20s - loss: 0.2968 - acc: 0.8782 - val_loss: 0.3535 - val_acc: 0.8568
Epoch 48/10000
97519/97519 [==============================] - 20s - loss: 0.2956 - acc: 0.8791 - val_loss: 0.3323 - val_acc: 0.8531
Epoch 49/10000
97440/97519 [============================>.] - ETA: 0sKF_10
Train on 97498 samples, validate on 13684 samples
Epoch 1/10000
97498/97498 [==============================] - 21s - loss: 0.5986 - acc: 0.7124 - va

97498/97498 [==============================] - 19s - loss: 0.2837 - acc: 0.8880 - val_loss: 0.3133 - val_acc: 0.8641
Epoch 56/10000
97498/97498 [==============================] - 20s - loss: 0.2832 - acc: 0.8882 - val_loss: 0.3762 - val_acc: 0.8394
Epoch 57/10000
97498/97498 [==============================] - 20s - loss: 0.2857 - acc: 0.8869 - val_loss: 0.2941 - val_acc: 0.8620
Epoch 58/10000
97440/97498 [============================>.] - ETA: 0s


In [10]:
print(model_name)

metrics_kf

2 CNN 2 AVG SM


,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
KF_1,0.953514,0.790878,0.790878,0.561482,0.701004,0.574330,0.857030,00:17:34.555242
KF_2,0.951207,0.817878,0.817878,0.556485,0.743536,0.553468,0.807181,00:18:55.627626
KF_3,0.948139,0.798526,0.798526,0.561029,0.712333,0.571702,0.848198,00:16:20.481742
KF_4,0.940560,0.801395,0.801395,0.560823,0.721445,0.569642,0.841226,00:09:54.524513
KF_5,0.948962,0.791670,0.794992,0.563831,0.703324,0.578466,0.861484,00:09:39.403508
KF_6,0.949881,0.801333,0.801333,0.565820,0.716151,0.580629,0.858590,00:09:35.970214
KF_7,0.957181,0.797340,0.800662,0.566381,0.719131,0.581319,0.858195,00:29:23.050171
KF_8,0.950442,0.771456,0.771456,0.546125,0.702664,0.536268,0.795975,00:15:58.446105
KF_9,0.948736,0.803425,0.806759,0.565245,0.717657,0.579430,0.856729,00:16:21.148333
KF_10,0.954753,0.782012,0.782012,0.561986,0.708297,0.574374,0.854098,00:19:30.975570


# Metrics

In [11]:
models

,AURoc Train,AURoc 0,AURoc 1,Precision,Recall,F1 score,Accuracy,Time
2 CNN 1 AVG RO,0.947529,0.745180,0.745180,0.553979,0.709121,0.556067,0.826375,00:11:31.092952
2 CNN 2 AVG RO,0.873747,0.801729,0.802394,0.544683,0.736958,0.512426,0.736872,00:15:31.066397
2 CNN 1 AVG SM,0.973189,0.768300,0.769298,0.565892,0.701109,0.581304,0.865456,00:10:26.555995
2 CNN 2 AVG SM,0.950338,0.795591,0.796589,0.560921,0.714554,0.569963,0.843871,00:16:19.418302
